## 1-Traitement des valeurs foncières

### A-Importation et Nettoyage des données

Première étape, on charge la librairie Pandas. 
On charge ensuite les 4 fichiers .txt correspondant aux données sur les valeurs foncières de  2018 à 2021.
On les regroupe en un seul dataframe 'df'.

In [2]:
import pandas as pd

df = pd.read_csv('./donnees/valeursfoncieres-2021.txt', sep="|", decimal=",")
df2 = pd.read_csv('./donnees/valeursfoncieres-2020.txt', sep="|", decimal=",")
df3 = pd.read_csv('./donnees/valeursfoncieres-2019.txt', sep="|", decimal=",")
df4 = pd.read_csv('./donnees/valeursfoncieres-2018.txt', sep="|", decimal=",")
df.head()


df = pd.concat([df, df2, df3, df4], axis=0, ignore_index=True)
df

C:\Users\marevel\AppData\Local\Temp\ipykernel_5468\993720581.py:3: DtypeWarning: Columns (18,23,24,26,28,30,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./donnees/valeursfoncieres-2021.txt', sep="|", decimal=",")
C:\Users\marevel\AppData\Local\Temp\ipykernel_5468\993720581.py:4: DtypeWarning: Columns (18,23,24,26,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./donnees/valeursfoncieres-2020.txt', sep="|", decimal=",")
C:\Users\marevel\AppData\Local\Temp\ipykernel_5468\993720581.py:5: DtypeWarning: Columns (18,23,24,26,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('./donnees/valeursfoncieres-2019.txt', sep="|", decimal=",")
C:\Users\marevel\AppData\Local\Temp\ipykernel_5468\993720581.py:6: DtypeWarning: Columns (14,18,23,24,26,28,32,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_c

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,97.0,5.0,S,NaN,2410.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,05/01/2021,Vente,...,NaN,0,3.0,Dépendance,NaN,0.0,0.0,S,NaN,2410.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2021,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,530.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,88.0,4.0,S,NaN,866.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,06/01/2021,Vente,...,NaN,0,1.0,Maison,NaN,168.0,5.0,S,NaN,1426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15125097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,28/12/2018,Vente,...,NaN,2,2.0,Appartement,NaN,150.0,4.0,NaN,NaN,NaN
15125098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/12/2018,Vente,...,NaN,1,2.0,Appartement,NaN,34.0,1.0,NaN,NaN,NaN
15125099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,28/12/2018,Vente,...,NaN,1,4.0,Local industriel. commercial ou assimilé,NaN,60.0,0.0,NaN,NaN,NaN
15125100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,13/12/2018,Adjudication,...,NaN,2,2.0,Appartement,NaN,54.0,2.0,NaN,NaN,NaN


'df' contient 15125102 lignes & 43 colonnes.

#### Suppression de colonnes vides

In [3]:
df_wo_NaN = df.dropna(axis=1, how='all') 

#### On supprimes les "doublons"